# Gasdata Calc
A simple demonstration of using [thermo](https://thermo.readthedocs.io/index.html) for gas processes.
In this demonstartion we are using one of the highest APIs thermo provide.

Thermo provides different equation of states. We use the orignal Peng Robinson EOS.

For the demonstration were are using the gas data provided from [Stadtwerke Hameln Weserbergland](https://www.stwhw-netze.de/fileadmin/user_upload/Brennwerte_Gasanalyse_Hameln_2022.pdf) 

In [30]:
# %pip install thermo
from thermo import EquilibriumStream, ChemicalConstantsPackage, CEOSGas, CEOSLiquid, PRMIX, FlashVL
from thermo.interaction_parameters import IPDB

In [31]:
# Input Gas Data
gas_data_mole_concentration = {
    'nitrogen':.910,
    'methane':89.871,
    'carbon-dioxide':1.825,
    'ethane':6.177,
    'propane':0.996,
    'iso-butane':0.091,
    'n-butane':0.103,
    'neo-pentane':.0,
    'iso-pentane':.015,
    'n-pentane':.012,
    'hydrogen':.0,
}

In [32]:
# use mole fraction
zs = [val/100 for val in gas_data_mole_concentration.values()]
assert round(sum(zs)) == 1

In [33]:
# prepare the flasher
constants, properties = ChemicalConstantsPackage.from_IDs(list(gas_data_mole_concentration))
kijs = IPDB.get_ip_asymmetric_matrix('ChemSep PR', constants.CASs, 'kij')
eos_kwargs = {'Pcs': constants.Pcs, 'Tcs': constants.Tcs, 'omegas': constants.omegas, 'kijs': kijs}
gas = CEOSGas(PRMIX, eos_kwargs=eos_kwargs, HeatCapacityGases=properties.HeatCapacityGases)
liquid = CEOSLiquid(PRMIX, eos_kwargs=eos_kwargs, HeatCapacityGases=properties.HeatCapacityGases)
flasher = FlashVL(constants, properties, liquid=liquid, gas=gas)

In [34]:
# generate streams at some given state
base_stream = EquilibriumStream(flasher, T=273.16, P=101300, zs=zs, m=1)

In [48]:
# now we compare with data from the labor
print(f"roh_mass: {base_stream.rho_mass():.3f}==0.802")
print(f"Wobbe Superior: {base_stream.Wobbe_index_normal()*2.77778e-7:.3f}==14.619")

roh_mass: 0.801==0.802
Wobbe Superior: 14.585==14.619


A common problem is the joule thomson coefficient

In [45]:
print(f"JT at base condition: {base_stream.Joule_Thomson()*1e5}")

JT at base condition: 0.6582631302978859


Now lets see if we can use the a state change

In [72]:
feed = EquilibriumStream(flasher, T=273.16+30, P=1.01e+7, zs=zs, m=20)
print(f"Feed Flow: {feed.Q*3600:.2f}")
# throttle
delta_pressure = 5.0e+6
product = EquilibriumStream(flasher, H=feed.H(), P=feed.P-delta_pressure, zs=zs, m=20)
print(f"Product Flow: {product.Q*3600:.2f}")
print(f"Delta T: {feed.T-product.T:.2f}")
print(f"Mean JT: {delta_pressure*1e-6 / (feed.T-product.T):.2f}")

Feed Flow: 806.97
Product Flow: 1559.12
Delta T: 22.44
Mean JT: 0.22
